In [ ]:
!pip install yahooquery

In [2]:
import pandas as pd
import yahooquery as yq

In [8]:
tick = 'aapl'

In [9]:
ticker = yq.Ticker(tick)
ticker

##Last Close Price

In [10]:
ticker_summary = ticker.summary_detail[tick]
ticker_summary.keys()

dict_keys(['maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield', 'exDividendDate', 'payoutRatio', 'fiveYearAvgDividendYield', 'beta', 'trailingPE', 'forwardPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 'twoHundredDayAverage', 'trailingAnnualDividendRate', 'trailingAnnualDividendYield', 'currency', 'fromCurrency', 'toCurrency', 'lastMarket', 'coinMarketCapLink', 'algorithm', 'tradeable'])

In [11]:
#Last Close Price
last_close_price = ticker_summary['previousClose']
last_close_price

179.58

##Operating Margin (%): Average of operating margins last 4 quarterly reports
$\text{Operation Margin} = \frac{\text{Operation Income}}{\text{Revenue}}$


In [22]:
op_fin = ['TotalOperatingIncomeAsReported', 'TotalRevenue']
op_margin_data = ticker.get_financial_data(op_fin,'q',trailing=False).iloc[-4:,:]

In [23]:
op_margin_data['OperationMargin'] = op_margin_data['TotalOperatingIncomeAsReported']/ op_margin_data['TotalRevenue']
op_margin_data

,asOfDate,periodType,currencyCode,TotalOperatingIncomeAsReported,TotalRevenue,OperationMargin
symbol,,,,,,
aapl,2022-06-30,3M,USD,2.307600e+10,8.295900e+10,0.278162
aapl,2022-09-30,3M,USD,2.489400e+10,9.014600e+10,0.276152
aapl,2022-12-31,3M,USD,3.601600e+10,1.171540e+11,0.307424
aapl,2023-03-31,3M,USD,2.831800e+10,9.483600e+10,0.298600


In [24]:
op_margin_data['OperationMargin'].mean()

0.2900844062823699

##EV / (EBITDA - Capex): Enterprise value / (EBITDA - Capital Expenditure) [from last annual financial statement]
$\text{EBITDA}=\text{EBIT}+\text{D&A}
\\\frac{\text{EV}}{(\text{EBITDA}-\text{CapEx})}=\frac{\text{EV}}{(\text{EBIT}+\text{D&A}-\text{CapEx})}$

In [97]:
last_ann_rep_date = ticker.income_statement(trailing=False)['asOfDate'][-1]

In [98]:
income_statement = ticker.income_statement(trailing=False)
income_statement = income_statement[income_statement['asOfDate']==last_ann_rep_date]
ebit = income_statement['EBIT'].values
ebit

array([1.19437e+11])

In [99]:
cashflow = ticker.cash_flow(trailing=False)[['asOfDate', 'CapitalExpenditure', 'DepreciationAndAmortization' ]]
cashflow = cashflow[cashflow['asOfDate']==last_ann_rep_date]
variable = pd.DataFrame(cashflow[['CapitalExpenditure', 'DepreciationAndAmortization']].values, index = cashflow['asOfDate'], columns = ['CapitalExpenditure', 'DepreciationAndAmortization' ])
variable['EBIT'] = ebit
variable['EBITDA'] =  variable['DepreciationAndAmortization'] + variable['EBIT']
variable['EBITDA-Capex'] = variable['EBITDA'] - variable['CapitalExpenditure']
variable

,CapitalExpenditure,DepreciationAndAmortization,EBIT,EBITDA,EBITDA-Capex
asOfDate,,,,,
2022-09-30,-1.070800e+10,1.110400e+10,1.194370e+11,1.305410e+11,1.412490e+11


In [102]:
ev = ticker.valuation_measures[['asOfDate','EnterpriseValue']]
ev = ev[ev['asOfDate']==last_ann_rep_date]['EnterpriseValue'].values
ev

array([2.27484134e+12])

In [106]:
ev_ebitda_capex_ratio = ev/variable['EBITDA-Capex'].values
ev_ebitda_capex_ratio

array([16.10518542])

##Revenue - 1 year annualized growth (%)

In [47]:
rev = ['TotalRevenue']
revenue = ticker.get_financial_data(rev,'a',trailing=False)
revenue['AnnalizedGrwoth'] = revenue['TotalRevenue'].pct_change()
revenue

,asOfDate,periodType,currencyCode,TotalRevenue,AnnalizedGrwoth
symbol,,,,,
aapl,2019-09-30,12M,USD,2.601740e+11,NaN
aapl,2020-09-30,12M,USD,2.745150e+11,0.055121
aapl,2021-09-30,12M,USD,3.658170e+11,0.332594
aapl,2022-09-30,12M,USD,3.943280e+11,0.077938


##Net Income - 1 year annualized growth (%)

In [48]:
ni = ['NetIncome']
net_income = ticker.get_financial_data(ni,'a',trailing=False)
net_income['AnnalizedGrwoth'] = net_income['NetIncome'].pct_change()
net_income

,asOfDate,periodType,currencyCode,NetIncome,AnnalizedGrwoth
symbol,,,,,
aapl,2019-09-30,12M,USD,5.525600e+10,NaN
aapl,2020-09-30,12M,USD,5.741100e+10,0.039000
aapl,2021-09-30,12M,USD,9.468000e+10,0.649161
aapl,2022-09-30,12M,USD,9.980300e+10,0.054109


##Short Interest
Short interest is the number of shorted shares divided by the number of shares outstanding.

In [12]:
ticker_key_stat = ticker.key_stats[tick]
ticker_key_stat.keys()

dict_keys(['maxAge', 'priceHint', 'enterpriseValue', 'forwardPE', 'profitMargins', 'floatShares', 'sharesOutstanding', 'sharesShort', 'sharesShortPriorMonth', 'sharesShortPreviousMonthDate', 'dateShortInterest', 'sharesPercentSharesOut', 'heldPercentInsiders', 'heldPercentInstitutions', 'shortRatio', 'shortPercentOfFloat', 'beta', 'impliedSharesOutstanding', 'category', 'bookValue', 'priceToBook', 'fundFamily', 'legalType', 'lastFiscalYearEnd', 'nextFiscalYearEnd', 'mostRecentQuarter', 'earningsQuarterlyGrowth', 'netIncomeToCommon', 'trailingEps', 'forwardEps', 'pegRatio', 'lastSplitFactor', 'lastSplitDate', 'enterpriseToRevenue', 'enterpriseToEbitda', '52WeekChange', 'SandP52WeekChange', 'lastDividendValue', 'lastDividendDate'])

In [14]:
short_interest = ticker_key_stat['sharesShort']*100/ticker_key_stat['sharesOutstanding']
short_interest

0.6985876016064619